In [ ]:
import json 
import pandas as pd
import sqlalchemy
import requests
import numpy as np
from datetime import date
from datetime import timedelta

  
def lambda_handler(event, context):
    schema="gans"
    host="name-of-your-aws-endpoint"
    user="admin"
    password="your-password-which-should-not-contain-@"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    
    request_flights_data(con)
    request_weather_data(con)
    
    return{
    	'statusCode': 200,
    	'body': json.dumps('Hello from Lambda!')
    }

def request_flights_data(con):
	API_key = "your-RapidAPI-key"
	tomorrow = date.today() + timedelta(days=1)
	from_local_time = f"{tomorrow}T00:00"
	to_local_time = f"{tomorrow}T12:00"
	arr_icao = "EDDB" # defines from which airport we extract arrival flights data
	arr_airport = "Berlin Brandenburg Airport"
	flights_dict = {
		'Flight_Number':[],
		'Departure_Airport': [], 
		'Departure_ICAO': [],
		'Arrival_Airport':[],
		'Arrival_ICAO': [],
		'Arrival_Terminal': [],
		'Arrival_Time': [], 
		'Status': [], 
		'Aircraft': [],
		'Airline':[]
		}
	headers = {
		"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
		"X-RapidAPI-Key": API_key
	}
	querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}
	url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{arr_icao}/{from_local_time}/{to_local_time}"
	response = requests.request("GET", url, headers=headers, params=querystring)
	flights = response.json()['arrivals']
	
	for l in flights:		
		# flight number
		try: flight_nr = l['number']
		except: flight_nr = None
		flights_dict['Flight_Number'].append(flight_nr)
		# departure airport
		try: dep_airport = l['departure']['airport']['name']
		except: dep_airport = None
		flights_dict['Departure_Airport'].append(dep_airport)
		# departure icao
		try: dep_icao = l['departure']['airport']['icao']
		except: dep_icao = None
		flights_dict['Departure_ICAO'].append(dep_icao)
		# arrival airport
		flights_dict['Arrival_Airport'].append(arr_airport)
		# arrival icao
		flights_dict['Arrival_ICAO'].append(arr_icao)
		# arrival terminal
		try: terminal = l['arrival']['terminal']
		except: terminal = None
		flights_dict['Arrival_Terminal'].append(terminal)
		# schedueled arrival time
		try: arr_time = l['arrival']['scheduledTimeLocal']
		except: arr_time = None
		flights_dict['Arrival_Time'].append(arr_time)
		# status of flight
		try: status = l['status']
		except: status= None
		flights_dict['Status'].append(status)
		# aircraft
		try: aircraft = l['aircraft']['model']
		except: aircraft = None
		flights_dict['Aircraft'].append(aircraft)
		# airline
		try: airline = l['airline']['name']
		except: airline = None
		flights_dict['Airline'].append(airline)
	
	flights_df = pd.DataFrame(flights_dict)
	flights_df['Arrival_Time'] = pd.to_datetime(flights_df['Arrival_Time'])
	(
		flights_df
		.replace({np.nan},'unknown')
	    .assign(Arrival_Time = lambda x: pd.to_datetime(x['Arrival_Time']))
	    .to_sql('arrivals', if_exists='append', con=con, index=False)
	    )
	

def request_weather_data(con):
	city= "Berlin"
	country = "DE" 
	API_key = "your-RapidAPI-key"
	url = "https://community-open-weather-map.p.rapidapi.com/forecast"
	headers = {
		"X-RapidAPI-Host": "community-open-weather-map.p.rapidapi.com",
		"X-RapidAPI-Key": API_key}
	querystring = {"q":f"{city},{country}"}
	
	response = requests.request("GET", url, headers=headers, params=querystring)
	forecast_api = response.json()['list']
	
	weather_info = []
	for forecast_3h in forecast_api: 
		weather_hour = {}
		# datetime utc
		weather_hour['datetime'] = forecast_3h['dt_txt']
		# temperature 
		weather_hour['temperature'] = forecast_3h['main']['temp']
		# wind
		weather_hour['wind'] = forecast_3h['wind']['speed']
		# probability precipitation 
		try: weather_hour['prob_perc'] = float(forecast_3h['pop'])
		except: weather_hour['prob_perc'] = 0
		# rain
		try: weather_hour['rain_qty'] = float(forecast_3h['rain']['3h'])
		except: weather_hour['rain_qty'] = 0
		# wind 
		try: weather_hour['snow'] = float(forecast_3h['snow']['3h'])
		except: weather_hour['snow'] = 0
		weather_hour['municipality_iso_country'] = city + ',' + country
		weather_info.append(weather_hour)
		# return(weather_info)
		
	weather_data = pd.DataFrame(weather_info)
	(
		weather_data
		.replace({np.nan},'unknown')
	    .assign(datetime = lambda x: pd.to_datetime(x['datetime']))
	    .to_sql('weather', if_exists='append', con=con, index=False))
	